In [ ]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex客户端库：使用具有在线预测训练管道的自定义训练文本二元分类模型

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/custom/showcase_custom_text_binary_classification_online_pipeline.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/custom/showcase_custom_text_binary_classification_online_pipeline.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程演示了如何使用Python的Vertex客户端库来训练和部署自定义文本二元分类模型以供在线预测，使用训练管道。

数据集

本教程使用的数据集是来自[TensorFlow Datasets](https://www.tensorflow.org/datasets/catalog/overview)的[IMDB电影评论](https://www.tensorflow.org/datasets/catalog/imdb_reviews)。在本教程中使用的数据集版本存储在一个公共云存储桶中。训练后的模型会预测评论是积极的还是消极的情感。

### 目标

在本教程中，您将使用 Vertex 客户端库在 Google 预构建的 Docker 容器中的 Python 脚本中创建一个自定义模型，然后通过发送数据对部署的模型进行预测。您也可以使用 `gcloud` 命令行工具或在线使用 Google Cloud Console 来创建自定义模型。

执行的步骤包括：

- 创建一个 Vertex 自定义作业以训练模型。
- 创建一个 `TrainingPipeline` 资源。
- 使用 `TrainingPipeline` 资源训练 TensorFlow 模型。
- 检索并加载模型工件。
- 查看模型评估。
- 将模型上传为 Vertex `Model` 资源。
- 将 `Model` 资源部署到服务的 `Endpoint` 资源。
- 进行预测。
- 取消部署 `Model` 资源。

成本

本教程使用Google Cloud（GCP）的收费组件：

* Vertex AI
* Cloud Storage

了解[Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing)和[Cloud Storage 价格](https://cloud.google.com/storage/pricing)，并使用[Pricing 计算器](https://cloud.google.com/products/calculator/)根据您的预期使用量生成成本估算。

安装

安装最新版本的Vertex客户库。

In [ ]:
import os
import sys

# Google Cloud Notebook
if os.path.exists("/opt/deeplearning/metadata/env_version"):
    USER_FLAG = "--user"
else:
    USER_FLAG = ""

! pip3 install -U google-cloud-aiplatform $USER_FLAG

安装最新的*google-cloud-storage*库的GA版本。

In [ ]:
! pip3 install -U google-cloud-storage $USER_FLAG

### 重新启动核心

一旦您安装了Vertex客户端库和Google的*云存储*，您需要重新启动笔记本核心，以便它可以找到这些软件包。

In [ ]:
if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

##开始前

###GPU运行时

*如果有这个选项，请确保在GPU运行时中运行此笔记本。在Colab中，选择* **运行时 > 更改运行时类型 > GPU**

###设置您的Google Cloud项目

**无论您使用的是哪种笔记本环境，以下步骤都是必需的。**

1. [选择或创建一个Google Cloud项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建账户时，您会获得$300的免费信用用于计算/存储成本。

2. [确保为您的项目启用了计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

3. [启用Vertex APIs和Compute Engine APIs。](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component)

4. [Google Cloud SDK](https://cloud.google.com/sdk)已经在Google Cloud Notebook中安装。

5. 在下面的单元格中输入您的项目ID。然后运行该单元格，以确保Cloud SDK为本笔记本中的所有命令使用正确的项目。

**注意**: Jupyter将以`!`为前缀的行视为shell命令，并将以`$`为前缀的Python变量插入这些命令中。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

区域

您还可以更改“REGION”变量，该变量用于整个笔记本的操作。以下是Vertex支持的区域。我们建议您选择离您最近的区域。

- 美洲：us-central1
- 欧洲：europe-west4
- 亚太地区：asia-east1

您不能使用多区域存储桶来进行Vertex训练。并非所有区域都支持所有Vertex服务。有关每个区域的最新支持，请参阅[Vertex位置文档](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

时间戳

如果您在进行实时教程会话，您可能正在使用共享测试账户或项目。为了避免在创建的资源上用户之间的名称冲突，您为每个实例会话创建一个时间戳，并附加在本教程中将要创建的资源的名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 验证您的 Google Cloud 帐户

**如果您正在使用 Google Cloud Notebook**，您的环境已经通过身份验证。跳过此步骤。

**如果您正在使用Colab**，运行下面的单元格，并在提示时按照说明进行身份验证。

**否则**，请按照以下步骤操作：

在 Cloud Console 中，转到[创建服务帐户密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

**单击创建服务帐户**。

在 **服务帐户名称** 字段中输入一个名称，并单击 **创建**。

在 **授予此服务帐户对项目的访问权限** 部分，单击角色下拉列表。在筛选框中输入“Vertex”，并选择 **Vertex 管理员**。在筛选框中输入“Storage Object Admin”，并选择 **Storage Object Admin**。

单击创建。包含您的密钥的 JSON 文件将下载到您的本地环境。

在下面的单元格中，将您的服务帐户密钥的路径输入为 GOOGLE_APPLICATION_CREDENTIALS 变量，然后运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# If on Google Cloud Notebook, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论您在哪个笔记本环境下，都需要遵循以下步骤。**

当您使用 Vertex 客户端库提交自定义训练作业时，您需要将包含训练代码的 Python 包上传到一个云存储桶中。Vertex 将从该包中运行代码。在本教程中，Vertex 还会将作业产生的训练模型保存在同一个存储桶中。然后，您可以根据此输出创建一个 `Endpoint` 资源，以便提供在线预测。

在下面设置您的云存储桶名称。存储桶名称在所有 Google Cloud 项目中必须是全局唯一的，包括您组织之外的项目。

In [ ]:
BUCKET_NAME = "gs://[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

只有如果您的存储桶尚不存在：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_NAME

最后，通过检查其内容验证对您的云存储桶的访问权限。

In [ ]:
! gsutil ls -al $BUCKET_NAME

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库并定义常量

#### 导入Vertex客户端库

将Vertex客户端库导入到我们的Python环境中。

In [ ]:
import time

from google.cloud.aiplatform import gapic as aip
from google.protobuf import json_format
from google.protobuf.json_format import MessageToJson, ParseDict
from google.protobuf.struct_pb2 import Struct, Value

顶点常量

为顶点设置以下常量：

- `API_ENDPOINT`：用于数据集、模型、作业、流水线和终端服务的顶点API服务端点。
- `PARENT`：用于数据集、模型、作业、流水线和终端资源的顶点位置根路径。

In [ ]:
# API service endpoint
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)

# Vertex location root path for your dataset, model and endpoint resources
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

#### CustomJob常量

设置CustomJob训练中的唯一常量：

- 数据集训练模式：告诉`Pipeline`资源服务为何任务（例如分类）训练模型。

In [ ]:
CUSTOM_TASK_GCS_PATH = (
    "gs://google-cloud-aiplatform/schema/trainingjob/definition/custom_task_1.0.0.yaml"
)

硬件加速器

设置用于训练和预测的硬件加速器（例如GPU）。

设置变量`TRAIN_GPU/TRAIN_NGPU`和`DEPLOY_GPU/DEPLOY_NGPU`以使用支持GPU的容器映像以及分配给虚拟机（VM）实例的GPU数量。例如，要在每个VM上分配4个Nvidia Telsa K80 GPU并使用GPU容器映像，您可以指定：

    （aip.AcceleratorType.NVIDIA_TESLA_K80，4）

对于GPU，可用的加速器包括：
   - aip.AcceleratorType.NVIDIA_TESLA_K80
   - aip.AcceleratorType.NVIDIA_TESLA_P100
   - aip.AcceleratorType.NVIDIA_TESLA_P4
   - aip.AcceleratorType.NVIDIA_TESLA_T4
   - aip.AcceleratorType.NVIDIA_TESLA_V100

否则，指定`(None, None)`以使用一个在CPU上运行的容器映像。

*注*: 在TF 2.3之前的版本中，对于GPU支持，将无法加载本教程中的自定义模型。这是一个已知问题，在TF 2.3中已修复 -- 这是由于生成在serving函数中的静态图操作造成的。如果您在自己的自定义模型上遇到此问题，请使用具有GPU支持的TF 2.3的容器映像。

In [ ]:
if os.getenv("IS_TESTING_TRAIN_GPU"):
    TRAIN_GPU, TRAIN_NGPU = (
        aip.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_TRAIN_GPU")),
    )
else:
    TRAIN_GPU, TRAIN_NGPU = (None, None)

if os.getenv("IS_TESTING_DEPOLY_GPU"):
    DEPLOY_GPU, DEPLOY_NGPU = (
        aip.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_DEPOLY_GPU")),
    )
else:
    DEPLOY_GPU, DEPLOY_NGPU = (None, None)

#### 容器（Docker）镜像

接下来，我们将设置用于训练和预测的Docker容器镜像

- TensorFlow 1.15
   - `gcr.io/cloud-aiplatform/training/tf-cpu.1-15:latest`
   - `gcr.io/cloud-aiplatform/training/tf-gpu.1-15:latest`
- TensorFlow 2.1
   - `gcr.io/cloud-aiplatform/training/tf-cpu.2-1:latest`
   - `gcr.io/cloud-aiplatform/training/tf-gpu.2-1:latest`
- TensorFlow 2.2
   - `gcr.io/cloud-aiplatform/training/tf-cpu.2-2:latest`
   - `gcr.io/cloud-aiplatform/training/tf-gpu.2-2:latest`
- TensorFlow 2.3
   - `gcr.io/cloud-aiplatform/training/tf-cpu.2-3:latest`
   - `gcr.io/cloud-aiplatform/training/tf-gpu.2-3:latest`
- TensorFlow 2.4
   - `gcr.io/cloud-aiplatform/training/tf-cpu.2-4:latest`
   - `gcr.io/cloud-aiplatform/training/tf-gpu.2-4:latest`
- XGBoost
   - `gcr.io/cloud-aiplatform/training/xgboost-cpu.1-1`
- Scikit-learn
   - `gcr.io/cloud-aiplatform/training/scikit-learn-cpu.0-23:latest`
- Pytorch
   - `gcr.io/cloud-aiplatform/training/pytorch-cpu.1-4:latest`
   - `gcr.io/cloud-aiplatform/training/pytorch-cpu.1-5:latest`
   - `gcr.io/cloud-aiplatform/training/pytorch-cpu.1-6:latest`
   - `gcr.io/cloud-aiplatform/training/pytorch-cpu.1-7:latest`

有关最新列表，请参阅[用于训练的预构建容器](https://cloud.google.com/vertex-ai/docs/training/pre-built-containers)。

- TensorFlow 1.15
   - `gcr.io/cloud-aiplatform/prediction/tf-cpu.1-15:latest`
   - `gcr.io/cloud-aiplatform/prediction/tf-gpu.1-15:latest`
- TensorFlow 2.1
   - `gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-1:latest`
   - `gcr.io/cloud-aiplatform/prediction/tf2-gpu.2-1:latest`
- TensorFlow 2.2
   - `gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-2:latest`
   - `gcr.io/cloud-aiplatform/prediction/tf2-gpu.2-2:latest`
- TensorFlow 2.3
   - `gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-3:latest`
   - `gcr.io/cloud-aiplatform/prediction/tf2-gpu.2-3:latest`
- XGBoost
   - `gcr.io/cloud-aiplatform/prediction/xgboost-cpu.1-2:latest`
   - `gcr.io/cloud-aiplatform/prediction/xgboost-cpu.1-1:latest`
   - `gcr.io/cloud-aiplatform/prediction/xgboost-cpu.0-90:latest`
   - `gcr.io/cloud-aiplatform/prediction/xgboost-cpu.0-82:latest`
- Scikit-learn
   - `gcr.io/cloud-aiplatform/prediction/sklearn-cpu.0-23:latest`
   - `gcr.io/cloud-aiplatform/prediction/sklearn-cpu.0-22:latest`
   - `gcr.io/cloud-aiplatform/prediction/sklearn-cpu.0-20:latest`

有关最新列表，请参阅[用于预测的预构建容器](https://cloud.google.com/vertex-ai/docs/predictions/pre-built-containers)。

In [ ]:
if os.getenv("IS_TESTING_TF"):
    TF = os.getenv("IS_TESTING_TF")
else:
    TF = "2-1"

if TF[0] == "2":
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf2-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf2-cpu.{}".format(TF)
else:
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf-cpu.{}".format(TF)

TRAIN_IMAGE = "gcr.io/cloud-aiplatform/training/{}:latest".format(TRAIN_VERSION)
DEPLOY_IMAGE = "gcr.io/cloud-aiplatform/prediction/{}:latest".format(DEPLOY_VERSION)

print("Training:", TRAIN_IMAGE, TRAIN_GPU, TRAIN_NGPU)
print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

#### 机器类型

接下来，设置用于训练和预测的机器类型。

- 设置变量`TRAIN_COMPUTE`和`DEPLOY_COMPUTE`来配置用于训练和预测的VM的计算资源。
 - `机器类型`
     - `n1-standard`：每个vCPU 3.75GB内存。
     - `n1-highmem`：每个vCPU 6.5GB内存。
     - `n1-highcpu`：每个vCPU 0.9GB内存。
 - `vCPUs`：\[2, 4, 8, 16, 32, 64, 96\]个处理器核心数。

*注意：以下情况不支持用于训练：*

 - `standard`：2个vCPUs。
 - `highcpu`：2、4和8个vCPUs。

*注意：您也可以使用n2和e2机器类型进行训练和部署，但它们不支持GPU。*

In [ ]:
if os.getenv("IS_TESTING_TRAIN_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_TRAIN_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
TRAIN_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", TRAIN_COMPUTE)

if os.getenv("IS_TESTING_DEPLOY_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_DEPLOY_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Deploy machine type", DEPLOY_COMPUTE)

# 教程

现在您已经准备好开始为IMDB电影评论创建自己的自定义模型并进行训练。

## 设置客户端

Vertex 客户端库采用客户端/服务器模型。在您这一侧（Python 脚本）您将创建一个客户端，该客户端向 Vertex 服务器发送请求并接收响应。

在本教程中，您将为工作流程中的不同步骤使用不同的客户端。因此，请提前设置它们。

- 用于 `Model` 资源的 Model 服务。
- 用于训练的 Pipeline 服务。
- 用于部署的 Endpoint 服务。
- 用于批处理作业和自定义训练的 Job 服务。
- 用于提供服务的 Prediction 服务。

In [ ]:
# client options same for all services
client_options = {"api_endpoint": API_ENDPOINT}


def create_model_client():
    client = aip.ModelServiceClient(client_options=client_options)
    return client


def create_pipeline_client():
    client = aip.PipelineServiceClient(client_options=client_options)
    return client


def create_endpoint_client():
    client = aip.EndpointServiceClient(client_options=client_options)
    return client


def create_prediction_client():
    client = aip.PredictionServiceClient(client_options=client_options)
    return client


clients = {}
clients["model"] = create_model_client()
clients["pipeline"] = create_pipeline_client()
clients["endpoint"] = create_endpoint_client()
clients["prediction"] = create_prediction_client()

for client in clients.items():
    print(client)

## 训练模型

有两种方法可以使用容器映像训练自定义模型：

- **使用 Google Cloud 预构建的容器**。如果使用预构建的容器，您还需要指定要安装到容器映像中的 Python 软件包。这个 Python 软件包包含了训练自定义模型的代码。

- **使用您自己的自定义容器映像**。如果使用自己的容器，容器需要包含训练自定义模型的代码。

准备您的自定义工作规范

现在您的客户已经准备好了，您的第一步是为您的自定义培训工作创建一个工作规范。工作规范将包括以下内容：

- `worker_pool_spec`：您将用于培训的机器类型和数量（单机或分布式）的规范
- `python_package_spec`：要安装在预构建容器中的Python包的规范

### 准备您的机器规格

现在为您的定制训练工作定义机器规格。这告诉 Vertex 需要为训练提供什么样的机器实例。
- `machine_type`：要提供的 GCP 实例的类型——例如，n1-standard-8。
- `accelerator_type`：硬件加速器的类型，如果有的话。在本教程中，如果您之前设置了变量`TRAIN_GPU != None`，则您正在使用 GPU；否则将使用 CPU。
- `accelerator_count`：加速器的数量。

In [ ]:
if TRAIN_GPU:
    machine_spec = {
        "machine_type": TRAIN_COMPUTE,
        "accelerator_type": TRAIN_GPU,
        "accelerator_count": TRAIN_NGPU,
    }
else:
    machine_spec = {"machine_type": TRAIN_COMPUTE, "accelerator_count": 0}

### 准备磁盘规格

（可选）现在为您的自定义训练作业定义磁盘规格。这告诉 Vertex 在每个训练机器实例中为训练提供什么类型和大小的磁盘。

- `boot_disk_type`: SSD 或 Standard。SSD更快，Standard更便宜。默认为SSD。
- `boot_disk_size_gb`: 磁盘大小（GB）。

In [ ]:
DISK_TYPE = "pd-ssd"  # [ pd-ssd, pd-standard]
DISK_SIZE = 200  # GB

disk_spec = {"boot_disk_type": DISK_TYPE, "boot_disk_size_gb": DISK_SIZE}

###定义工作池规范

接下来，您需要定义自定义训练作业的工作池规范。工作池规范将包括以下内容：

- `replica_count`：要提供的此机器类型的实例数量。
- `machine_spec`：硬件规范。
- `disk_spec`：（可选）磁盘存储规范。

- `python_package`：要安装在VM实例上的Python训练包，以及要调用的Python模块，以及Python模块的命令行参数。

现在让我们更深入地了解Python包规范：

- `executor_image_spec`：这是为您的自定义训练作业配置的docker镜像。

- `package_uris`：这是要安装在提供的实例上的Python训练包的位置（URI）列表。这些位置需要在Cloud Storage存储桶中。这些可以是单独的Python文件或整个包的zip（归档）。在后一种情况下，作业服务将解压（解档）内容到docker镜像中。

- `python_module`：要调用以运行自定义训练作业的Python模块（脚本）。在本例中，您将调用 `trainer.task.py` -- 请注意，没有必要附加 `.py` 后缀。

- `args`：要传递给相应Python模块的命令行参数。在本例中，您将设置：
  - `"--model-dir=" + MODEL_DIR`：存储模型工件的Cloud Storage位置。告知训练脚本保存模型工件的两种方式有：
      - 直接：将Cloud Storage位置作为命令行参数传递给训练脚本（设置变量 `DIRECT = True`），
      - 间接：服务将Cloud Storage位置作为环境变量 `AIP_MODEL_DIR` 传递给训练脚本（设置变量 `DIRECT = False`）。在这种情况下，您通过工作规范告知服务模型工件的位置。
  - `"--epochs=" + EPOCHS`：训练的纪元数量。
  - `"--steps=" + STEPS`：每个纪元的步骤（批次）数量。
  - `"--distribute=" + TRAIN_STRATEGY"`：用于单个或分布式训练的训练分发策略。
     - `"single"`：单个设备。
     - `"mirror"`：单个计算实例上的所有GPU设备。
     - `"multi"`：所有计算实例上的所有GPU设备。

In [ ]:
JOB_NAME = "custom_job_" + TIMESTAMP
MODEL_DIR = "{}/{}".format(BUCKET_NAME, JOB_NAME)

if not TRAIN_NGPU or TRAIN_NGPU < 2:
    TRAIN_STRATEGY = "single"
else:
    TRAIN_STRATEGY = "mirror"

EPOCHS = 20
STEPS = 100

DIRECT = True
if DIRECT:
    CMDARGS = [
        "--model-dir=" + MODEL_DIR,
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
        "--distribute=" + TRAIN_STRATEGY,
    ]
else:
    CMDARGS = [
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
        "--distribute=" + TRAIN_STRATEGY,
    ]

worker_pool_spec = [
    {
        "replica_count": 1,
        "machine_spec": machine_spec,
        "disk_spec": disk_spec,
        "python_package_spec": {
            "executor_image_uri": TRAIN_IMAGE,
            "package_uris": [BUCKET_NAME + "/trainer_imdb.tar.gz"],
            "python_module": "trainer.task",
            "args": CMDARGS,
        },
    }
]

### 检查培训包

#### 包布局

在开始培训之前，您将查看如何为自定义培训任务组装Python包。解压缩后，包含以下目录/文件布局的包。

- PKG-INFO
- README.md
- setup.cfg
- setup.py
- trainer
  - \_\_init\_\_.py
  - task.py

文件 `setup.cfg` 和 `setup.py` 是将包安装到 Docker 镜像的操作环境的指令。

文件 `trainer/task.py` 是执行自定义培训任务的 Python 脚本。*注意*，当我们在工作池规范中引用它时，我们将目录斜杠替换为点号 (`trainer.task`)，并去掉文件后缀名 (`.py`)。

#### 包装配

在接下来的单元格中，您将组装培训包。

In [ ]:
# Make folder for Python training script
! rm -rf custom
! mkdir custom

# Add package information
! touch custom/README.md

setup_cfg = "[egg_info]\n\ntag_build =\n\ntag_date = 0"
! echo "$setup_cfg" > custom/setup.cfg

setup_py = "import setuptools\n\nsetuptools.setup(\n\n    install_requires=[\n\n        'tensorflow_datasets==1.3.0',\n\n    ],\n\n    packages=setuptools.find_packages())"
! echo "$setup_py" > custom/setup.py

pkg_info = "Metadata-Version: 1.0\n\nName: IMDB Movie Reviews text binary classification\n\nVersion: 0.0.0\n\nSummary: Demostration training script\n\nHome-page: www.google.com\n\nAuthor: Google\n\nAuthor-email: aferlitsch@google.com\n\nLicense: Public\n\nDescription: Demo\n\nPlatform: Vertex"
! echo "$pkg_info" > custom/PKG-INFO

# Make the training subfolder
! mkdir custom/trainer
! touch custom/trainer/__init__.py

#### Task.py 文件内容

在下一个单元格中，您将编写训练脚本 task.py 的内容。我不会详细介绍，这只是让您浏览一下而已。总结如下：

- 从命令行 (`--model_dir`) 中获取保存模型工件的目录，如果未指定，则从环境变量 `AIP_MODEL_DIR` 中获取。
- 从 TF Datasets (tfds) 中加载 IMDB 电影评论数据集。
- 使用 TF.Keras 模型 API 构建一个简单的 RNN 模型。
- 编译模型（`compile()`）。
- 根据参数 `args.distribute` 设置训练分发策略。
- 使用指定的 epochs 训练模型（`fit()`）。
- 将训练完成的模型保存到指定的模型目录中（`save(args.model_dir)`）。

In [ ]:
%%writefile custom/trainer/task.py
# Single, Mirror and Multi-Machine Distributed Training for IMDB

import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.python.client import device_lib
import argparse
import os
import sys
tfds.disable_progress_bar()

parser = argparse.ArgumentParser()
parser.add_argument('--model-dir', dest='model_dir',
                    default=os.getenv('AIP_MODEL_DIR'), type=str, help='Model dir.')
parser.add_argument('--lr', dest='lr',
                    default=1e-4, type=float,
                    help='Learning rate.')
parser.add_argument('--epochs', dest='epochs',
                    default=20, type=int,
                    help='Number of epochs.')
parser.add_argument('--steps', dest='steps',
                    default=100, type=int,
                    help='Number of steps per epoch.')
parser.add_argument('--distribute', dest='distribute', type=str, default='single',
                    help='distributed training strategy')
args = parser.parse_args()

print('Python Version = {}'.format(sys.version))
print('TensorFlow Version = {}'.format(tf.__version__))
print('TF_CONFIG = {}'.format(os.environ.get('TF_CONFIG', 'Not found')))
print(device_lib.list_local_devices())

# Single Machine, single compute device
if args.distribute == 'single':
    if tf.test.is_gpu_available():
        strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
    else:
        strategy = tf.distribute.OneDeviceStrategy(device="/cpu:0")
# Single Machine, multiple compute device
elif args.distribute == 'mirror':
    strategy = tf.distribute.MirroredStrategy()
# Multiple Machine, multiple compute device
elif args.distribute == 'multi':
    strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()

# Multi-worker configuration
print('num_replicas_in_sync = {}'.format(strategy.num_replicas_in_sync))

# Preparing dataset
BUFFER_SIZE = 10000
BATCH_SIZE = 64


def make_datasets():
    dataset, info = tfds.load('imdb_reviews/subwords8k', with_info=True,
                          as_supervised=True)
    train_dataset, test_dataset = dataset['train'], dataset['test']
    encoder = info.features['text'].encoder

    padded_shapes = ([None],())
    return train_dataset.shuffle(BUFFER_SIZE).padded_batch(BATCH_SIZE, padded_shapes), encoder


train_dataset, encoder = make_datasets()

# Build the Keras model
def build_and_compile_rnn_model(encoder):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(encoder.vocab_size, 64),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
        optimizer=tf.keras.optimizers.Adam(args.lr),
        metrics=['accuracy'])
    return model


with strategy.scope():
  # Creation of dataset, and model building/compiling need to be within
  # `strategy.scope()`.
  model = build_and_compile_rnn_model(encoder)

# Train the model
model.fit(train_dataset, epochs=args.epochs, steps_per_epoch=args.steps)
model.save(args.model_dir)

将培训脚本存储在您的云存储桶中

接下来，您将培训文件夹打包为一个压缩的tar球，然后将其存储在您的云存储桶中。

In [ ]:
! rm -f custom.tar custom.tar.gz
! tar cvf custom.tar custom
! gzip custom.tar
! gsutil cp custom.tar.gz $BUCKET_NAME/trainer_imdb.tar.gz

## 使用 `TrainingPipeline` 资源训练模型

现在开始使用 Vertex 上的训练管道训练您的自定义训练作业。要训练您的自定义模型，请执行以下步骤：

1. 为 `Dataset` 资源创建一个 Vertex `TrainingPipeline` 资源。
2. 执行管道开始训练。

### 创建 `TrainingPipeline` 资源

您可能会问，我们为什么要使用管道？通常当作业（如训练）有多个步骤时，我们通常会按顺序使用管道：执行步骤 A，执行步骤 B，等等。通过将步骤放入管道中，我们获得以下好处：

1. 可以在后续训练作业中重复使用。
2. 可以作为批处理作业容器化并运行。
3. 可以进行分布式处理。
4. 所有步骤与同一管道作业相关联，以跟踪进度。

#### `training_pipeline` 规范

首先，您需要描述一个管道规范。让我们看看构建自定义作业的 `training_pipeline` 规范的 *最少* 要求：

- `display_name`: 管道作业的可读名称。
- `training_task_definition`: 训练任务模式。
- `training_task_inputs`: 描述训练作业要求的字典。
- `model_to_upload`: 描述（上传的）Vertex 自定义 `Model` 资源规范的字典。
  - `display_name`: `Model` 资源的可读名称。
  - `artificat_uri`: 保存模型工件以 SavedModel 格式存储的 Cloud Storage 路径。
  - `container_spec`: 这是要安装到 `Endpoint` 资源上的 Docker 容器的规范，从中自定义模型将服务预测。

In [ ]:
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value

MODEL_NAME = "custom_pipeline-" + TIMESTAMP
PIPELINE_DISPLAY_NAME = "custom-training-pipeline" + TIMESTAMP

training_task_inputs = json_format.ParseDict(
    {"workerPoolSpecs": worker_pool_spec}, Value()
)
pipeline = {
    "display_name": PIPELINE_DISPLAY_NAME,
    "training_task_definition": CUSTOM_TASK_GCS_PATH,
    "training_task_inputs": training_task_inputs,
    "model_to_upload": {
        "display_name": PIPELINE_DISPLAY_NAME + "-model",
        "artifact_uri": MODEL_DIR,
        "container_spec": {"image_uri": DEPLOY_IMAGE},
    },
}

print(pipeline)

#### 创建训练流水线

使用这个辅助函数 `create_pipeline`，它接受以下参数：

- `training_pipeline`：管道训练作业的完整规范。

该辅助函数调用管道客户端服务的 `create_pipeline` 方法，该方法接受以下参数：

- `parent`：您的 `Dataset`、`Model` 和 `Endpoint` 资源的 Vertex 位置根路径。
- `training_pipeline`：管道训练作业的完整规范。

该辅助函数将返回分配给训练流水线的 Vertex 完全限定标识符，保存为 `pipeline.name`。

In [ ]:
def create_pipeline(training_pipeline):

    try:
        pipeline = clients["pipeline"].create_training_pipeline(
            parent=PARENT, training_pipeline=training_pipeline
        )
        print(pipeline)
    except Exception as e:
        print("exception:", e)
        return None
    return pipeline


response = create_pipeline(pipeline)

现在保存你创建的训练流水线的唯一标识符。

In [ ]:
# The full unique ID for the pipeline
pipeline_id = response.name
# The short numeric ID for the pipeline
pipeline_short_id = pipeline_id.split("/")[-1]

print(pipeline_id)

### 获取培训管道的信息

现在只获取此培训管道实例的管道信息。帮助函数通过调用作业客户服务的`get_training_pipeline`方法，传入以下参数来获取此作业的作业信息：

- `name`：Vertex完全限定的管道标识符。

当模型训练完成时，管道状态将是`PIPELINE_STATE_SUCCEEDED`。

In [ ]:
def get_training_pipeline(name, silent=False):
    response = clients["pipeline"].get_training_pipeline(name=name)
    if silent:
        return response

    print("pipeline")
    print(" name:", response.name)
    print(" display_name:", response.display_name)
    print(" state:", response.state)
    print(" training_task_definition:", response.training_task_definition)
    print(" training_task_inputs:", dict(response.training_task_inputs))
    print(" create_time:", response.create_time)
    print(" start_time:", response.start_time)
    print(" end_time:", response.end_time)
    print(" update_time:", response.update_time)
    print(" labels:", dict(response.labels))
    return response


response = get_training_pipeline(pipeline_id)

部署

对上述模型进行训练可能需要超过20分钟的时间。

一旦您的模型训练完成，您可以通过将`end_time`减去`start_time`来计算训练模型所需的实际时间。对于您的模型，您需要知道完全限定的Vertex模型资源标识符，即管道服务分配给它的。您可以从返回的管道实例中获取此信息，字段为`model_to_deploy.name`。

In [ ]:
while True:
    response = get_training_pipeline(pipeline_id, True)
    if response.state != aip.PipelineState.PIPELINE_STATE_SUCCEEDED:
        print("Training job has not completed:", response.state)
        model_to_deploy_id = None
        if response.state == aip.PipelineState.PIPELINE_STATE_FAILED:
            raise Exception("Training Job Failed")
    else:
        model_to_deploy = response.model_to_upload
        model_to_deploy_id = model_to_deploy.name
        print("Training Time:", response.end_time - response.start_time)
        break
    time.sleep(60)

print("model to deploy:", model_to_deploy_id)

In [ ]:
if not DIRECT:
    MODEL_DIR = MODEL_DIR + "/model"
model_path_to_deploy = MODEL_DIR

加载保存的模型

您的模型以TensorFlow SavedModel格式存储在一个云存储桶中。现在从云存储桶加载它，然后您可以做一些事情，比如评估模型和进行预测。

要加载模型，您可以使用TF.Keras的`model.load_model()`方法，将保存模型的云存储路径（由`MODEL_DIR`指定）传递给它。

In [ ]:
import tensorflow as tf

model = tf.keras.models.load_model(MODEL_DIR)

## 评估模型

现在让我们来评估模型的表现如何。

### 加载评估数据

您将使用`load()`方法从`tfds.datasets`加载IMDB电影评论测试（留置）数据。这将返回一个包含两个元素的元组，第一个元素是数据集，第二个是数据集的信息，其中包含预定义的词汇编码器。编码器会将单词转换为数字嵌入，这些嵌入是预先训练并在自定义训练脚本中使用的。

在训练模型时，需要为文本设置一个固定的输入长度。对于前向传递的批处理，相应的`tf.dataset`的`padded_batch()`属性被设置为将每个输入序列填充为相同形状的批处理。

对于测试数据，您也需要相应地设置`padded_batch()`属性。

In [ ]:
import tensorflow_datasets as tfds

dataset, info = tfds.load("imdb_reviews/subwords8k", with_info=True, as_supervised=True)
test_dataset = dataset["test"]
encoder = info.features["text"].encoder

BATCH_SIZE = 64
padded_shapes = ([None], ())
test_dataset = test_dataset.padded_batch(BATCH_SIZE, padded_shapes)

### 执行模型评估

现在评估自定义作业中的模型表现如何。

In [ ]:
model.evaluate(test_dataset)

## 上传模型以进行服务

接下来，您将从自定义作业将您的TF.Keras模型上传到Vertex `Model`服务，这将为您的自定义模型创建一个Vertex `Model`资源。在上传过程中，您需要定义一个服务函数来将数据转换为您的模型预期的格式。如果您向Vertex发送编码数据，则您的服务函数将确保在将数据作为输入传递给您的模型之前，在模型服务器上对数据进行解码。

### 服务函数如何工作

当您向在线预测服务器发送请求时，请求将由一个HTTP服务器接收。HTTP服务器从HTTP请求内容主体中提取预测请求。提取的预测请求将转发到服务函数。对于Google预构建的预测容器，请求内容将作为`tf.string`传递给服务函数。

服务函数由两部分组成：

- `预处理函数`：
  - 将输入（`tf.string`）转换为底层模型（动态图）所期望的输入形状和数据类型。
  - 执行与训练底层模型期间相同的数据预处理--例如，归一化，缩放等。
- `后处理函数`：
  - 将模型输出转换为接收应用程序所期望的格式--例如，压缩输出。
  - 为接收应用程序打包输出--例如，添加标题，制作JSON对象等。

预处理和后处理函数都转换为静态图，然后与模型融合。底层模型的输出传递给后处理函数。后处理函数将转换/打包的输出传送回HTTP服务器。HTTP服务器将输出作为HTTP响应内容返回。

在为TF.Keras模型构建服务函数时，需要考虑一个问题，即它们作为静态图运行。这意味着您不能使用需要动态图的TF图操作。如果您这样做，将在服务函数的编译期间收到错误，指示您正在使用不受支持的EagerTensor。

获取服务功能签名

您可以通过重新加载模型到内存，并查询每个层对应的签名来获取您模型的输入和输出层的签名。

在进行预测请求时，您需要将请求路由到服务功能而不是模型，因此您需要知道服务功能的输入层名称 -- 这将在您进行预测请求时使用。

In [ ]:
loaded = tf.saved_model.load(model_path_to_deploy)

serving_input = list(
    loaded.signatures["serving_default"].structured_input_signature[1].keys()
)[0]
print("Serving function input:", serving_input)

### 上传模型

使用这个辅助函数`upload_model`来上传您的模型，以SavedModel格式存储，到`Model`服务中，该服务会为您的模型实例化一个Vertex `Model`资源实例。一旦您完成了这一步，您可以像任何其他Vertex `Model`资源实例一样使用`Model`资源实例，比如部署到用于提供预测的`Endpoint`资源。

辅助函数接受以下参数：

- `display_name`：`Endpoint`服务的可读名称。
- `image_uri`：模型部署的容器镜像。
- `model_uri`：我们的SavedModel工件的Cloud Storage路径。对于本教程，这是`trainer/task.py`保存模型工件的Cloud Storage位置，我们在变量`MODEL_DIR`中指定。

该辅助函数调用`Model`客户端服务的`upload_model`方法，该方法接受以下参数：

- `parent`：`Dataset`、`Model`和`Endpoint`资源的Vertex位置根路径。
- `model`：Vertex `Model`资源实例的规范。

现在让我们更深入地了解Vertex模型规范`model`。这是一个字典对象，包括以下字段：

- `display_name`：`Model`资源的可读名称。
- `metadata_schema_uri`：由于您的模型是没有使用Vertex `Dataset`资源构建的，因此您将其留空（`''`）。
- `artifact_uri`：保存模型以SavedModel格式存储的Cloud Storage路径。
- `container_spec`：这是将安装在`Endpoint`资源上的Docker容器的规范，`Model`资源将从中提供预测。使用您之前设置的变量`DEPLOY_GPU != None`来使用GPU；否则只分配CPU。

将模型上传到Vertex Model资源会返回一个长时间运行的操作，因为可能需要一些时间。您调用response.result()，这是一个同步调用，将在Vertex Model资源准备就绪时返回。

辅助函数返回相应的Vertex Model实例`upload_model_response.model`的完全限定标识符。您将保存该标识符以备后续步骤中使用，保存在变量`model_to_deploy_id`中。

In [ ]:
IMAGE_URI = DEPLOY_IMAGE


def upload_model(display_name, image_uri, model_uri):
    model = {
        "display_name": display_name,
        "metadata_schema_uri": "",
        "artifact_uri": model_uri,
        "container_spec": {
            "image_uri": image_uri,
            "command": [],
            "args": [],
            "env": [{"name": "env_name", "value": "env_value"}],
            "ports": [{"container_port": 8080}],
            "predict_route": "",
            "health_route": "",
        },
    }
    response = clients["model"].upload_model(parent=PARENT, model=model)
    print("Long running operation:", response.operation.name)
    upload_model_response = response.result(timeout=180)
    print("upload_model_response")
    print(" model:", upload_model_response.model)
    return upload_model_response.model


model_to_deploy_id = upload_model("imdb-" + TIMESTAMP, IMAGE_URI, model_path_to_deploy)

获取“Model”资源信息

现在让我们仅获取您的模型的模型信息。使用这个辅助函数“get_model”，参数如下：

- `name`：指定“Model”资源的Vertex唯一标识符。

这个辅助函数调用了Vertex“Model”客户端服务的方法“get_model”，参数如下：

- `name`：指定“Model”资源的Vertex唯一标识符。

In [ ]:
def get_model(name):
    response = clients["model"].get_model(name=name)
    print(response)


get_model(model_to_deploy_id)

部署`Model`资源

现在部署训练好的Vertex自定义`Model`资源。这需要两个步骤：

1. 为部署`Model`资源创建一个`Endpoint`资源。

2. 将`Model`资源部署到`Endpoint`资源中。

### 创建 `Endpoint` 资源

使用此辅助函数 `create_endpoint` 来创建一个用于部署模型以提供预测的端点，具有以下参数：

- `display_name`: 用于 `Endpoint` 资源的人类可读名称。

该辅助函数使用端点客户端服务的 `create_endpoint` 方法，该方法接受以下参数：

- `display_name`: 用于 `Endpoint` 资源的人类可读名称。

创建一个 `Endpoint` 资源会返回一个长时间运行的操作，因为可能需要一些时间来为提供服务的 `Endpoint` 资源进行配置。您可以调用 `response.result()`，这是一个同步调用，将在 `Endpoint` 资源准备就绪时返回。辅助函数将返回 `Endpoint` 资源的 Vertex 完全限定标识符：`response.name`。

In [ ]:
ENDPOINT_NAME = "imdb_endpoint-" + TIMESTAMP


def create_endpoint(display_name):
    endpoint = {"display_name": display_name}
    response = clients["endpoint"].create_endpoint(parent=PARENT, endpoint=endpoint)
    print("Long running operation:", response.operation.name)

    result = response.result(timeout=300)
    print("result")
    print(" name:", result.name)
    print(" display_name:", result.display_name)
    print(" description:", result.description)
    print(" labels:", result.labels)
    print(" create_time:", result.create_time)
    print(" update_time:", result.update_time)
    return result


result = create_endpoint(ENDPOINT_NAME)

现在获取您创建的“终端点”资源的唯一标识符。

In [ ]:
# The full unique ID for the endpoint
endpoint_id = result.name
# The short numeric ID for the endpoint
endpoint_short_id = endpoint_id.split("/")[-1]

print(endpoint_id)

### 计算实例扩展

处理在线预测请求时，您有几种选择来扩展计算实例：

- 单个实例：在线预测请求在单个计算实例上处理。
  - 将计算实例的最小（`MIN_NODES`）和最大（`MAX_NODES`）数量设置为一。

- 手动扩展：在线预测请求在您手动指定的固定数量的计算实例上分配。
  - 将计算实例的最小（`MIN_NODES`）和最大（`MAX_NODES`）数量设置为相同的节点数量。当模型首次部署到实例时，将预配固定数量的计算实例，并在其上均匀分配在线预测请求。

- 自动扩展：在线预测请求在可扩展数量的计算实例上分配。
  - 将计算实例的最小（`MIN_NODES`）数量设置为模型首次部署时预配和取消预配的数量，并根据负载条件将最大（`MAX_NODES`）数量的计算实例设置为预配。

计算实例的最小数量对应于字段`min_replica_count`，最大数量对应于字段`max_replica_count`，在您后续的部署请求中。

In [ ]:
MIN_NODES = 1
MAX_NODES = 1

### 将 `Model` 资源部署到 `Endpoint` 资源

使用这个辅助函数 `deploy_model` 将 `Model` 资源部署到您为提供预测创建的 `Endpoint` 资源，使用以下参数:

- `model`: 要从训练管道上传（部署）的 Vertex 完全限定模型标识符。
- `deploy_model_display_name`: 部署模型的人类可读名称。
- `endpoint`: 部署模型的 Vertex 完全限定端点标识符。

辅助函数调用 `Endpoint` 客户端服务的方法 `deploy_model`，该方法采用以下参数：

- `endpoint`: 部署 `Model` 资源到的 Vertex 完全限定 `Endpoint` 资源标识符。
- `deployed_model`: 部署模型的要求规范。
- `traffic_split`: 在端点上发送到此模型的流量百分比，表示为一个或多个键/值对的字典。
   - 如果只有一个模型，则指定为 **{ "0": 100 }**，其中 "0" 指的是上传的模型，100 表示流量的 100%。
   - 如果端点上有现有的模型，需要分担流量，则使用 `model_id` 来指定为 **{ "0": 百分比, model_id: 百分比, ... }**，其中 `model_id` 是要部署到的端点的现有模型的模型标识符。百分比必须加起来等于 100。

现在让我们深入了解 `deployed_model` 参数。此参数被指定为一个具有最小必填字段的 Python 字典：

- `model`: 要上传的模型的 Vertex 完全限定模型标识符。
- `display_name`: 部署模型的人类可读名称。
- `disable_container_logging`: 这将禁用容器事件的记录，例如执行失败（默认情况下容器日志记录已启用）。在部署调试时通常启用容器日志记录，然后在部署生产时禁用。
- `dedicated_resources`: 这指的是为了提供预测请求而扩展的计算实例（副本）的数量。
  - `machine_spec`: 要创建的计算实例。使用您之前设置的变量 `DEPLOY_GPU != None`，以使用 GPU；否则仅分配一个 CPU。
  - `min_replica_count`: 首次创建的计算实例数量，您之前设置的变量 `MIN_NODES`。
  - `max_replica_count`: 要扩展到的最大计算实例数量，您之前设置的变量 `MAX_NODES`。

#### 流量分配

现在让我们深入了解 `traffic_split` 参数。此参数被指定为一个 Python 字典。这可能一开始会有些困惑。让我解释一下，您可以将模型的多个实例部署到一个端点，然后设置每个实例接收多少（百分比）。

为什么要这样做？也许您已经在生产环境中部署了先前版本 -- 让我们称之为 v1。您在 v2 上获得了更好的模型评估，但在将其部署到生产环境之前，您不确定它是否真的更好。因此，在流量分配的情况下，您可能希望将 v2 部署到与 v1 相同的端点，但它只占流量的10%。这样，您可以在不干扰大多数用户的情况下监视其表现，直到做出最终决定。

#### 响应

该方法返回一个长时间运行的操作 `response`。我们将通过调用 `response.result()` 同步等待操作完成，这将阻塞直到模型部署完毕。如果这是首次将模型部署到该端点，则可能需要额外几分钟来完成资源的创建。

In [ ]:
DEPLOYED_NAME = "imdb_deployed-" + TIMESTAMP


def deploy_model(
    model, deployed_model_display_name, endpoint, traffic_split={"0": 100}
):

    if DEPLOY_GPU:
        machine_spec = {
            "machine_type": DEPLOY_COMPUTE,
            "accelerator_type": DEPLOY_GPU,
            "accelerator_count": DEPLOY_NGPU,
        }
    else:
        machine_spec = {
            "machine_type": DEPLOY_COMPUTE,
            "accelerator_count": 0,
        }

    deployed_model = {
        "model": model,
        "display_name": deployed_model_display_name,
        "dedicated_resources": {
            "min_replica_count": MIN_NODES,
            "max_replica_count": MAX_NODES,
            "machine_spec": machine_spec,
        },
        "disable_container_logging": False,
    }

    response = clients["endpoint"].deploy_model(
        endpoint=endpoint, deployed_model=deployed_model, traffic_split=traffic_split
    )

    print("Long running operation:", response.operation.name)
    result = response.result()
    print("result")
    deployed_model = result.deployed_model
    print(" deployed_model")
    print("  id:", deployed_model.id)
    print("  model:", deployed_model.model)
    print("  display_name:", deployed_model.display_name)
    print("  create_time:", deployed_model.create_time)

    return deployed_model.id


deployed_model_id = deploy_model(model_to_deploy_id, DEPLOYED_NAME, endpoint_id)

发出一个在线预测请求

现在对部署的模型进行在线预测。

### 准备请求内容

由于数据集是一个`tf.dataset`，它充当生成器的角色，我们必须将其用作迭代器来访问测试数据中的数据项。我们按照以下步骤从测试数据中获取单个数据项：

- 使用`take(1)`方法将每次迭代要获取的批次数量属性设置为一。
- 通过测试数据进行一次迭代，即在for循环中执行break。
- 在单次迭代中，我们保存数据项，其形式为一个元组。
- 数据项将是元组的第一个元素，然后您将其从张量转换为numpy数组 -- `data[0].numpy()`。

In [ ]:
import tensorflow_datasets as tfds

dataset, info = tfds.load("imdb_reviews/subwords8k", with_info=True, as_supervised=True)
test_dataset = dataset["test"]

test_dataset.take(1)
for data in test_dataset:
    print(data)
    break

test_item = data[0].numpy()

### 发送预测请求

好的，现在您有一个测试数据项。使用以下辅助函数`predict_data`，它接受以下参数：

- `data`: 测试数据项是一个经过64填充的numpy 1D数组。
- `endpoint`: 模型部署在的 Vertex AI 的完全限定标识符。
- `parameters_dict`: 用于服务的附加参数。

此函数使用预测客户端服务并调用`predict`方法，参数如下:

- `endpoint`: 模型部署在的 Vertex AI 的完全限定标识符。
- `instances`: 要预测的实例（数据项）的列表。
- `parameters`: 用于服务的附加参数。

要将测试数据传递给预测服务，您必须将其打包以供发送到服务的二进制文件，步骤如下：

    1. 将数据项从1D numpy数组转换为1D Python列表。
    2. 将预测请求转换为序列化的 Google protobuf (`json_format.ParseDict()`)

预测请求中的每个实例都是以下形式的字典项：

                        {输入名称: 内容}
                          
- `输入名称`: 底层模型的输入层的名称。
- `内容`: 作为1D Python列表的数据项。

由于`predict()`服务可以处理多个数据项（实例），您将单个数据项发送为一个数据项的列表。作为最后一步，将实例列表打包成Google的protobuf格式——这是我们传递给`predict()`服务的内容。

`response`对象返回一个列表，其中列表中的每个元素对应请求中的相应图像。每个预测的输出中您将看到：

- `predictions` -- 在0（负面）和1（积极）之间的预测二进制情绪。

In [ ]:
def predict_data(data, endpoint, parameters_dict):
    parameters = json_format.ParseDict(parameters_dict, Value())

    # The format of each instance should conform to the deployed model's prediction input schema.
    instances_list = [{serving_input: data.tolist()}]
    instances = [json_format.ParseDict(s, Value()) for s in instances_list]

    response = clients["prediction"].predict(
        endpoint=endpoint, instances=instances, parameters=parameters
    )
    print("response")
    print(" deployed_model_id:", response.deployed_model_id)
    predictions = response.predictions
    print("predictions")
    for prediction in predictions:
        print(" prediction:", prediction)


predict_data(test_item, endpoint_id, None)

## 卸载 `Model` 资源

现在从服务的 `Endpoint` 资源中卸载你的 `Model` 资源。使用这个辅助函数 `undeploy_model`，它接受以下参数：

- `deployed_model_id`：当 `Model` 资源被部署到时由终端服务返回的模型部署标识符。
- `endpoint`：`Model` 部署到的 `Endpoint` 资源的 Vertex 完全限定标识符。

该函数调用终端客户端服务的方法 `undeploy_model`，带有以下参数：

- `deployed_model_id`：终端服务在部署 `Model` 资源时返回的模型部署标识符。
- `endpoint`：`Model` 资源部署到的 `Endpoint` 资源的 Vertex 完全限定标识符。
- `traffic_split`：如何在 `Endpoint` 资源上的剩余部署模型中分割流量。

由于这是 `Endpoint` 资源上唯一部署的模型，你可以简单地通过将其设置为空来留空 `traffic_split`。

In [ ]:
def undeploy_model(deployed_model_id, endpoint):
    response = clients["endpoint"].undeploy_model(
        endpoint=endpoint, deployed_model_id=deployed_model_id, traffic_split={}
    )
    print(response)


undeploy_model(deployed_model_id, endpoint_id)

清理

为了清理这个项目中使用的所有GCP资源，您可以删除用于本教程的GCP项目。

否则，您可以删除在本教程中创建的个别资源：

- 数据集
- 流水线
- 模型
- 端点
- 批处理作业
- 自定义作业
- 超参数调整作业
- 云存储桶

In [ ]:
delete_dataset = True
delete_pipeline = True
delete_model = True
delete_endpoint = True
delete_batchjob = True
delete_customjob = True
delete_hptjob = True
delete_bucket = True

# Delete the dataset using the Vertex fully qualified identifier for the dataset
try:
    if delete_dataset and "dataset_id" in globals():
        clients["dataset"].delete_dataset(name=dataset_id)
except Exception as e:
    print(e)

# Delete the training pipeline using the Vertex fully qualified identifier for the pipeline
try:
    if delete_pipeline and "pipeline_id" in globals():
        clients["pipeline"].delete_training_pipeline(name=pipeline_id)
except Exception as e:
    print(e)

# Delete the model using the Vertex fully qualified identifier for the model
try:
    if delete_model and "model_to_deploy_id" in globals():
        clients["model"].delete_model(name=model_to_deploy_id)
except Exception as e:
    print(e)

# Delete the endpoint using the Vertex fully qualified identifier for the endpoint
try:
    if delete_endpoint and "endpoint_id" in globals():
        clients["endpoint"].delete_endpoint(name=endpoint_id)
except Exception as e:
    print(e)

# Delete the batch job using the Vertex fully qualified identifier for the batch job
try:
    if delete_batchjob and "batch_job_id" in globals():
        clients["job"].delete_batch_prediction_job(name=batch_job_id)
except Exception as e:
    print(e)

# Delete the custom job using the Vertex fully qualified identifier for the custom job
try:
    if delete_customjob and "job_id" in globals():
        clients["job"].delete_custom_job(name=job_id)
except Exception as e:
    print(e)

# Delete the hyperparameter tuning job using the Vertex fully qualified identifier for the hyperparameter tuning job
try:
    if delete_hptjob and "hpt_job_id" in globals():
        clients["job"].delete_hyperparameter_tuning_job(name=hpt_job_id)
except Exception as e:
    print(e)

if delete_bucket and "BUCKET_NAME" in globals():
    ! gsutil rm -r $BUCKET_NAME